In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

In [2]:
import os
# Set JAVA_HOME to the correct Java installation path
os.environ['JAVA_HOME'] = '/opt/homebrew/Cellar/openjdk@17/17.0.17/libexec/openjdk.jdk/Contents/Home'
# Create SparkSession
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()
data_path = '/Users/zlatavorobeva/Documents/GitHub/SQL Int/SQL_Intermediate/Spark/Data'

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/22 19:37:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/22 19:37:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/22 19:37:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/12/22 19:37:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/12/22 19:37:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
json_df2_path = data_path + "/utilization.json"
df_util = spark.read.format("json").load(json_df2_path)

In [4]:
vectorAssembler = VectorAssembler(inputCols=["cpu_utilization", "free_memory", "session_count"], outputCol="features")

In [5]:
vcluster_df = vectorAssembler.transform(df_util)

In [6]:
vcluster_df.show()

+---------------+-------------------+-----------+---------+-------------+----------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|
+---------------+-------------------+-----------+---------+-------------+----------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.35,58.0]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58| [0.41,0.4,58.0]|

In [7]:
kmeans = KMeans().setK(3)
kmeans = kmeans.setSeed(1)

In [8]:
kmodel = kmeans.fit(vcluster_df)

25/12/22 19:40:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [9]:
kmodel.clusterCenters()

[array([ 0.52047775,  0.47836303, 51.79927162]),
 array([ 0.62881549,  0.37094643, 70.43030159]),
 array([ 0.71931575,  0.28104316, 88.23965784])]